# Mass Flow Meter \& Datalogger Scripts

## General Technology

Mass flow meters are widely used instruments in fluid mechanics and engineering applications to measure the mass or volume flow rate of a fluid. The fundamental working principle involves a **low-inertia turbine** placed in the fluid flow path. As the fluid passes through the turbine, it rotates at a speed proportional to the flow rate. 

### Key Components and Operation
1. **Turbine with Magnetic Ring**: The turbine has a magnetic ring attached to it, with one or more interruptions (gaps) in the magnetic field.
2. **Hall Effect Sensor**: A sensor positioned near the turbine detects the changes in the magnetic field as the turbine rotates.
3. **Pulse Counting**: The sensor generates a series of electrical pulses, where the frequency of the pulses corresponds to the rotational speed of the turbine.
4. **Meter Constant (K-Factor)**: The turbine and sensor assembly has a meter constant (K-factor), defined as the number of pulses generated per unit of flow. This constant is used to convert the pulse frequency into a mass or volume flow rate.

The output is then processed by a microcontroller or data acquisition system to display the flow rate in appropriate units.

### Advantages
- **Accuracy**: Low-inertia turbines minimize lag, allowing precise measurement even at low flow rates.
- **Compact and Scalable**: Mass flow meters are available in a wide range of sizes and capacities, suitable for different applications.
- **Digital Output**: The pulse-based output is easy to integrate with modern control systems and data loggers.

## Common Engineering Applications

Mass flow meters are versatile and widely used across industries. Some common applications include:

### 1. **Water and Wastewater Treatment**
   - Monitoring the flow of water, effluent, and chemicals in treatment plants.
   - Ensuring compliance with regulatory discharge limits.

### 2. **Oil and Gas Industry**
   - Measuring fuel flow rates in pipelines.
   - Monitoring the consumption of lubricants and other fluids in mechanical systems.

### 3. **HVAC Systems**
   - Ensuring accurate flow rates of chilled water, refrigerants, and steam in heating and cooling systems.
   - Optimizing energy efficiency by monitoring flow conditions.

### 4. **Food and Beverage Industry**
   - Measuring the flow of liquids like milk, juice, and syrup in processing plants.
   - Monitoring the flow of cleaning agents during sterilization processes.

### 5. **Automotive and Aerospace Engineering**
   - Fuel consumption testing for engines and turbines.
   - Flow monitoring in hydraulic and pneumatic systems.

### 6. **Pharmaceutical and Chemical Manufacturing**
   - Precise dosing of chemicals in production lines.
   - Monitoring the flow of solvents and active ingredients in formulation processes.

## Summary

Mass flow meters offer a reliable and precise solution for measuring fluid flow rates across a broad spectrum of engineering applications. Their ability to provide near real-time digital data and integrate seamlessly with automation systems makes them indispensable in modern industrial processes.

## Lab Setup: Raspberry Pi Zero W as Datalogger and Controller

In this lab, we will use a **Raspberry Pi Zero W** as the datalogger and controller, paired with an **inexpensive turbine-type mass flow meter**. The Raspberry Pi Zero W is preconfigured as a WiFi host to simplify connectivity and data acquisition.

### Steps to Access the Datalogger
1. **Connect to the Raspberry Pi's WiFi**:
   - Locate the SSID provided in class and connect your device to it.
   - No internet access is required, as this connection is solely for interfacing with the Raspberry Pi.

2. **Access the Raspberry Pi**:
   - Use an SSH client to connect to the Raspberry Pi's IP address (provided in class).
   - Once connected via SSH, start the VNC server using the command:
     ```bash
     vncserver
     ```

3. **Open the VNC Desktop**:
   - Use a VNC client to connect to the Raspberry Pi's desktop environment.
   - This provides full access to the Raspberry Pi's graphical interface.

4. **Run the Mass Flow Meter Program**:
   - Locate the program file (`/path/filename`, as provided in class) on the Raspberry Pi.
   - Start the program using a terminal command:
     ```bash
     python3 /path/filename
     ```
   - When prompted, supply:
     - **Meter Constant (K-Factor)**: Provided with the mass flow meter.
     - **Desired Time Interval**: The time in seconds for each reading.

### Input Validation
The program is designed with robust input validation using Python's `try-except` blocks to handle invalid or pathological entries, ensuring smooth operation and accurate data logging.

With this setup, the Raspberry Pi Zero W enables efficient and flexible data acquisition from the turbine-type mass flow meter, offering a hands-on experience with both modern IoT technology and traditional fluid mechanics instrumentation.

## Python Scripts for Flow Measurement Lab

### Overview
Below are Python scripts provided to document the workflow and enhance the understanding of the lab. These scripts are designed for data logging, processing, and visualization.

### Script 1: Datalogger Initialization
**Purpose**:  
This script establishes communication between the Raspberry Pi and the turbine flow meter. It allows users to input key parameters, such as:
- **Meter Constant (K-Factor)**: Provided with the flow meter.
- **Sampling Interval**: The time between successive measurements.

Once running, the script writes flow rate data to the console, you can use redirection to write to a file if you wish.
:::{warning}
The script needs modification to prompt for the meter constant and time interval.  Use Try-Except structure to reduce pathalogical inputs
:::

```python
#!/usr/bin/env python
# Code to record Hall Detector events and convert into flow rate
# by T.G. Cleveland 6 FEB 2019
# Base Code from : https://raspberrypi.stackexchange.com/questions/34480
# This code will operate two different flow meters, modify if only using a single meter
#
# Tested (no sensor) 6 FEB 2019 - exit keyboard OK
#                               - syntax check OK
#
#                    5 JUN 2019 - modify for fixed run length
#                               - fixed run length OK
#                               - modify for multiple sensors
#
#                   11 MAR 2022 - refactor to append to an output file for
#                               - near real-time reporting
import RPi.GPIO as GPIO
import time,sys
# set sensor pin
flow_sensor_one = 23
flow_sensor_two = 24
#
# configure GPIO 
GPIO.setmode(GPIO.BCM)
GPIO.setup(flow_sensor_one, GPIO.IN, pull_up_down = GPIO.PUD_UP)
GPIO.setup(flow_sensor_two, GPIO.IN, pull_up_down = GPIO.PUD_UP)
#
# initialize counter(s)
#
global count
global count2
count = 0
count2 = 0
#
# define a counter method
#
def countPulse(channel):
   global count
   if start_counter == 1:
      count = count + 1
def countPulse2(channel):
   global count2
   if start_counter2 == 1:
      count2 = count2 + 1
#   now = time.strftime("%c") # capture actual time as center of sampling interval
#   print count
#   flow = count / (60.0 * 7.5) # force to float 
#   print(now + ' ' + str(count) + ' ' + str(flow))
#   print(now)
#
# 
GPIO.add_event_detect(flow_sensor_one, GPIO.FALLING, callback=countPulse)
GPIO.add_event_detect(flow_sensor_two, GPIO.FALLING, callback=countPulse2)
# Read meter specifications file

Afile = open("meterspecs.txt","r") # open meterspecs.txt connection
stationID = Afile.readline()
sensorID = Afile.readline()
howmany2wait = int(Afile.readline())
howmany2read = int(Afile.readline())
meterconstant = float(Afile.readline())
meterconstant2 = float(Afile.readline())
filename = Afile.readline().rstrip('\n')
Afile.close() # disconnect the specifications file
accCount = 0
accCount2 = 0
howmanyRread = 0
# set up header for the output
print('filenme',filename)
print('# Flowmeter Recording System ')
print('# ' + stationID),
print('# ' + sensorID),
print('# Sampling Interval Duration (seconds) ' + str(howmany2wait))
print('# Sampling Interval Count (Count * Duration = Elaped Time) ' + str(howmany2read))
print('# Events is number of counted pulses for a sampling interval')
print('# Volume is computed volume in liters for a sampling interval')
print('# 1 and 2 are the two sampling channels; Pin 16 & 18 (GPIO 23 & 24)')
print('DateTime' + ', Events1' + ', Events2 ' + ', Volume1 ' + ', Volume2 ')
#externalfile=open(filename,'a') # output file to append to
# run the datalogger for howmany2read minutes
while True:
   try:
      if howmanyRread >= howmany2read :
         print '\ntime to die!'
         print(' Total Events1 = ' + str(accCount) + ' Total Events2 = ' + str(accCount2))
         print(' Total Volume1 = ' + str(meterconstant*accCount) + ' Total Volume2 = ' + str(meterconstant2*accCount2))
         enow = time.strftime("%a %b %d %Y") # capture system time
         externalfile=open(filename,'a') # output file to append to
         externalfile.write('#Totals ' + enow + ' Events1 = ' + str(accCount) + ' Events2 = ' + str(accCount2) + '\n')
         externalfile.write('#Totals ' + enow + ' Volume1 = ' + str(meterconstant*accCount) + ' Volume2 = ' + str(meterconstant2*accCount2) + '\n')
         externalfile.close()
	 GPIO.cleanup()
         sys.exit()
      howmanyRread = howmanyRread + 1
      start_counter = 1
      start_counter2 = 1
      time.sleep(howmany2wait)
      start_counter = 0
      start_counter2 = 0
#flow = (count * 60 * 2.25 / 1000)
      flow  = (count *  meterconstant  )
      flow2 = (count2 * meterconstant2 )
      now = time.strftime("%c") # capture system time
      print(now + ' , ' + str(count) + ' , ' + str(count2) + ' , ' + str(flow) + ' , ' + str(flow2))
      externalfile=open(filename,'a') # output file to append to
      externalfile.write(now + ' , ' + str(count) + ' , ' + str(count2) + ' , ' + str(flow) + ' , ' + str(flow2) + '\n')
      externalfile.close()      
      accCount = accCount + count
      accCount2 = accCount2 + count2
      count = 0
      count2 = 0
      time.sleep(1)
   except KeyboardInterrupt:
      print '\ncaught keyboard interrupt!, bye'
      print(' Total Events1 = ' + str(accCount) + ' Total Events2 = ' + str(accCount2))
      print(' Total Volume1 = ' + str(meterconstant*accCount) + ' Total Volume2 = ' + str(meterconstant2*accCount2))
      externalfile.close()     
      GPIO.cleanup()
      sys.exit()

 

```
### Script 2: Alert-Type Flow Monitoring
This script establishes communication between the Raspberry Pi and the turbine flow meter. The script counts pulses over an interval and then checks if the count exceeds a limit. If it does, it sends an email. This script uses some of the `pigpio` code for doing the counts.

The script listed below is verbatim from [https://github.com/ravra/waterFlowMeter](https://github.com/ravra/waterFlowMeter)

:::{warning}
The script needs to be able to send the email to the recipiet, so some testing and modification is needed - don't use as is.  It is stored here for a future semester.
:::

```python
#!/usr/bin/env python

# Simple script to monitor a cheap water flow sensor attached to gpio 4. The script counts pulses over an interval and then
# checks if the count exceeds a limit. If it does, it sends an email.
# This script uses some of the pigpio code for doing the counts.
# ravraid 1/31/19

import time, datetime
from smtplib import SMTP_SSL as SMTP
from email.MIMEText import MIMEText
import pigpio

intervalTime  = 15  # in seconds
triggerMin    = 50  # limit in pulse counts - adjust to detect small leaks

SMTPserver    = 'smtp.gmail.com'

sender        = 'FOO@BLAH.COM'
destination   = ['FOO@BLAH.COM']
USERNAME      = "FOO@BLAH.COM"
PASSWORD      = "MYPASSWORD"

subject       = ""
waterFlow     = 0
flowGpio      = 4

text_subtype = 'plain'

def sendMail(content):
    try:
        msg = MIMEText(content, text_subtype)
        msg['Subject']=       subject
        msg['From']   = sender # some SMTP servers will do this automatically, not all

        conn = SMTP(SMTPserver)
        conn.set_debuglevel(False)
        conn.login(USERNAME, PASSWORD)
        try:
            conn.sendmail(sender, destination, msg.as_string())
        finally:
            conn.close()

    except Exception, exc:
        sys.exit( "mail failed; %s" % str(exc) ) # give a error message

pi = pigpio.pi()

pi.set_mode(flowGpio, pigpio.INPUT)
pi.set_pull_up_down(flowGpio, pigpio.PUD_DOWN)

flowCallback = pi.callback(flowGpio, pigpio.FALLING_EDGE)

old_count   = 0
triggerTime = datetime.datetime.today() - datetime.timedelta(weeks=1)  # Initialize it to more than a day ago

while True:

   time.sleep(intervalTime)

   count = flowCallback.tally()
   waterFlow = count - old_count
   #print("counted {} pulses".format(waterFlow))
   yesterday = datetime.datetime.today() - datetime.timedelta(days=1)
   # Only send at most one message per day so check if the last trigger was more than 24hours ago
   if ( (waterFlow > triggerMin) & (triggerTime < yesterday) ):
       triggerTime = datetime.datetime.today()
       print "Note: Sending mail.."
       subject= "Waterflow over limit: " + str(waterFlow)
       message= "Limit is: " + str(triggerMin) + " and water flow is: " + str(waterFlow) + '\n'
       message = message + '\n\n' + "Sent from rpiZero1 waterFlowMeter.py" + '\n'
       sendMail(message)
   old_count = count

pi.stop()
```
<!--### Script 2: Data Processing and Visualization
**Purpose**:  
This script processes the logged data and generates useful insights, including:
- Summary statistics (e.g., average, max, and min flow rates).
- Calibration curves for the flow meter.
- Time-series plots of flow rate data.

The script also demonstrates how to handle and visualize data effectively using Python libraries like `pandas` and `matplotlib`.-->


## End of Section